# **BackToTheAttention**

This project aims to use the attention mechanism for time series prediction. As time series data needs special handling and I had no experience in doing so, I decided to use one of the most beautiful, influential, and genius mechanisms ever to combine it with stock prediction. I will try to predict stocks for Pinterest (ticker: "PINS"), but one can reuse this code for any stock.

This notebook, as well as its successor: `inference.ipynb`, follows a **code-first** structure, meaning I first implemented the code under the `src/` directory and then simply copy-pasted the code here, so some aspects might have been implemented differently if the project were notebook-oriented.

**Note**: Since this script uses Yahoo Finance's API to get the stock data, it needs an internet connection to run. Although installing the libraries also requires an internet connection, that might not be too project-specific.

First, let's start with library installation, imports, and logger setup:

In [1]:
!pip install scikit-learn torch yfinance
import argparse
import joblib
import logging
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d/%m/%Y %H:%M:%S')
logger = logging.getLogger("BackToTheTrainLogger (Train)")

## **Data Acquisition**

We have to define a method for acquiring the data for any stock and custom period:

In [2]:
def get_data(ticker: str, period: str = "3y") -> pd.DataFrame:
    data = yf.Ticker(ticker)
    df = data.history(period=period)  # only the latest 'period' stocks
    logger.info("HISTORY:\n%s\n%s", list(df.columns), df.head())  # we want to predict the 'Close' price

    # we don't really care about dividends and stock splits, as they provide no information related to the 'Close' price (most of the time they are 0).
    # I am not a quant, but this is based on my limited knowledge; therefore, we drop those two columns and use the rest for the task.
    df.drop(columns=["Dividends", "Stock Splits"], inplace=True)

    return df

MinMax scaling is necessary for this task because neural network based architectures, including Transformers, train more stably and converge faster when input features are normalized to a consistent range like $[0,1]$ rather than using raw stock prices that can vary widely in magnitude. Large or varying input values can lead to exploding or vanishing gradients during backpropagation, particularly in attention mechanisms where dot products between queries and keys become unbalanced and dominate the softmax unfairly. Scaling also ensures that the model's learned patterns are driven by relative changes in price rather than absolute levels, which is especially helpful for financial time series.

I fit `MinMaxScaler` (0 to 1) ONLY on training portion to prevent data leakage.
$$x_\text{scaled} = \frac{x - \text{train}_{\text{min}}}{\text{train}_{\text{max}} - \text{train}_{\text{min}}}$$

In [3]:
def minmax_scale(series: np.ndarray, train_ratio: float = 0.8) -> tuple[np.ndarray, np.ndarray, MinMaxScaler]:
    n = len(series)
    split_idx = int(n * train_ratio)  # chronological split point
    train_series = series[:split_idx]
    test_series = series[split_idx:]

    # fit scaler only on train data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(train_series)
    train_scaled = scaler.transform(train_series)
    test_scaled = scaler.transform(test_series)

    logger.info("Train scaled shape: %s, Test scaled shape: %s", train_scaled.shape, test_scaled.shape)
    logger.info("Scaler min: %.4f, scale: %.4f", scaler.min_[0], scaler.scale_[0])

    return train_scaled, test_scaled, scaler

Define the `create_windows` (sliding window) method, which for $X_i = [x_{i}, x_{i + 1}, \cdots, x_{i + L - 1}] \in \mathbb{R}^{L \times 1}$ and $y_i = x_{i + L}$ returns $X \in \mathbb{R}^{N \times L \times 1}, y \in \mathbb{R}^{N \times 1}$ where $N =$ `len(scaled_data)`, a.k.a. lookback.

In [4]:
def create_windows(scaled_data: np.ndarray, L: int = 30) -> tuple[np.ndarray, np.ndarray]:
    if len(scaled_data) <= L:
        raise ValueError("Data length %d must be > lookback %d", len(scaled_data), L)

    X, y = [], []
    for i in range(len(scaled_data) - L):
        X.append(scaled_data[i:i + L])
        y.append(scaled_data[i + L])

    X = np.array(X)  # shape: (N, L, 1)
    y = np.array(y).squeeze(-1)  # shape: (N,) for easier loss computation

    logger.info("Created windows -> X shape: %s, y shape: %s", X.shape, y.shape)

    return X, y

before moving on to the architecture, I finally define:

In [5]:
def prepare_datasets(df: pd.DataFrame, L: int = 30, train_ratio: float = 0.8) -> tuple[
    np.ndarray, np.ndarray, np.ndarray, np.ndarray, MinMaxScaler]:
    series = df['Close'].values.reshape(-1, 1)
    train_scaled, test_scaled, scaler = minmax_scale(series, train_ratio=train_ratio)

    # test windows use ONLY past test data (no train leakage)
    X_train, y_train = create_windows(train_scaled, L=L)
    X_test, y_test = create_windows(test_scaled, L=L)

    logger.info("Final datasets ready -> Train samples: %d, Test samples: %d", len(X_train), len(X_test))

    return X_train, y_train, X_test, y_test, scaler

## **Embedding**

I read the paper about the Rotary Positional Embedding in August 2025, and as it is better than simple absolute or relative embeddings, why not use it? RoPE is a mixture of absolute positional embedding, which was introduced in the original "Attention is All You Need" paper (where the base $10000$ comes from), and relative encoding (which is not really efficient to compute, as opposed to absolute and rotary). Because of the latter, I thought it would be a great fit for the time-series data.

In [6]:
class RoPE(nn.Module):
    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        self.dim = dim
        self.base = base

        # precompute frequencies: \theta_i = base^{-2i/dim}
        inv_freq = 1.0 / (base ** (torch.arange(start=0, end=dim, step=2, dtype=torch.float32) / dim))

        # to save as part of a model's persistent state without treating it as a learnable parameter
        self.register_buffer(name="inv_freq", tensor=inv_freq, persistent=True)  # shape: (dim / 2,)

    def forward(self, x: torch.Tensor, seq_len: int) -> torch.Tensor:
        positions = torch.arange(0, seq_len, device=x.device, dtype=self.inv_freq.dtype)  # (seq_len,)
        angles = positions[:, None] * self.inv_freq[None, :]  # (seq_len, dim//2)
        sin, cos = torch.sin(angles), torch.cos(angles)  # (seq_len, dim / 2)
        # expand dims for broadcasting: (seq_len, dim//2) -> (1, seq_len, dim//2)
        sin = sin.unsqueeze(0)
        cos = cos.unsqueeze(0)

        # split the last dimension
        x1 = x[..., : self.dim // 2]  # (B, seq_len, dim / 2)
        x2 = x[..., self.dim // 2:]  # (B, seq_len, dim / 2)
        # Rotation matrix
        rotated_x1 = x1 * cos - x2 * sin
        rotated_x2 = x1 * sin + x2 * cos

        return torch.cat([rotated_x1, rotated_x2], dim=-1)

## **DeLorean + Attention = DeTention**

`DeTentionBlock` is the core repeating unit that applies RoPE-enhanced self-attention and feed-forward processing with residual connections. I won't be repeating what arguments this class takes, as that's obvious, but here is my though process for the architecture:

Apply embedding to the normalized layer, then compute the attention with RoPE-embedded $q$ and $k$ (not $v$!), normalize again and combine with the original input to allow the gradient flow, then use feed-forward neural net to figure out the "final" token.

**Note**: as we use GLU, it requires `ff_hidden_size` to be of even dimension. If you do not want to have such restriction, maybe try replacing GLU with GELU.

In [7]:
class DeTentionBlock(nn.Module):
    def __init__(self, d_model: int, n_heads: int, ff_hidden_size: int = 256, dropout: float = 0.2):
        super().__init__()
        self.d_model = d_model  # must be even and divisible by n_heads
        self.n_heads = n_heads
        self.ff_hidden_size = ff_hidden_size
        self.dropout = dropout

        self.norm1 = nn.LayerNorm(d_model)
        self.rope = RoPE(dim=d_model, base=10000.0)
        self.attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=n_heads, dropout=dropout, batch_first=True)  # makes input (batch, seq, dim) instead of (seq, batch, dim)
        self.norm2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_hidden_size),
            nn.GLU(dim=-1),  # https://arxiv.org/pdf/2002.05202; please, check the last line of section 4 (Conclusions)
            nn.Dropout(dropout),
            nn.Linear(ff_hidden_size // 2, d_model)
        )
        self.dropout_layer = nn.Dropout(dropout)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Attention sub-layer with residual connection
        residual = x
        x = self.norm1(x)
        q, k = self.rope(x, seq_len=x.shape[1]), self.rope(x, seq_len=x.shape[1])
        attn_output, _ = self.attention(query=q, key=k, value=x, need_weights=False)  # I do not intend to visualize attention
        x = self.dropout_layer(attn_output) + residual

        # FF sub-layer with residual connection
        residual = x
        x = self.ff(self.norm2(x))
        x = self.dropout_layer(x) + residual

        return x

`DeTention`, which is the complete end-to-end model that handles input projection from raw prices, stacks multiple `DeTention` blocks and produces the final prediction. It has raw stock prices to $\mathbb{R}^{d_\text{model}}$ projection, then uses $k$ `DeTentionBlock`s and optional average pooling for final prediction.

In [8]:
class DeTention(nn.Module):
    def __init__(self, seq_len: int = 30, d_model: int = 64, n_heads: int = 4, n_layers: int = 2, ff_hidden_size: int = 256, dropout: float = 0.2, use_avg_pool: bool = True):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        self.use_avg_pool = use_avg_pool

        self.input_proj = nn.Linear(1, d_model)  # 'Close' -> d_model
        self.DeTention_blocks = nn.ModuleList([DeTentionBlock(d_model=d_model, n_heads=n_heads, ff_hidden_size=ff_hidden_size, dropout=dropout) for i in range(n_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.output_head = nn.Linear(d_model, 1)

        # optional global average pooling instead of last token
        if use_avg_pool:
            self.pool = nn.AdaptiveAvgPool1d(output_size=1)  # (B, d_model, L) -> (B, d_model, 1)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.input_proj(x)  # (batch_size, seq_len, 1) -> (batch_size, seq_len, d_model)
        for block in self.DeTention_blocks:
            x = block(x)
        x = self.norm(x)

        # pooling or last token
        if self.use_avg_pool:
            x = self.pool(x.transpose(1, 2)).squeeze(-1)  # (B, d_model, L) -> (B, d_model); nn.AdaptiveAvgPool1d expects channel dimension second (like images: N, C, L)
        else:
            x = x[:, -1, :]  # last position

        x = self.output_head(x)  # next price prediction: (batch_size, 1)

        return x.squeeze(dim=-1)  # (batch_size,)

We also have methods for saving and loading the `DeTention` model:

In [9]:
def save_model(model: DeTention, path: str = "models/DeTention.pth") -> None:  # save the entire model (architecture + weights)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save({
        'state_dict': model.state_dict(),
        'config': {
            'seq_len': model.seq_len,
            'd_model': model.d_model,
            'use_avg_pool': model.use_avg_pool
        }
    }, path)


def load_model(path: str = "models/DeTention.pth", **model_kwargs) -> DeTention:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Model file not found: {path}")

    checkpoint = torch.load(path, map_location=torch.device('cpu'), weights_only=True)
    config = checkpoint.get('config', {})
    config.update(model_kwargs)
    model = DeTention(**config)
    model.load_state_dict(checkpoint['state_dict'])

    return model

## Setup for training

First, let's declare hyperparameters:

In [10]:
ticker = "PINS"
period = "3y"
L = 30
train_ratio = 0.8
batch_size = 32
lr = 1e-4
epochs = 100
patience = 15
d_model = 64
n_heads = 4
n_layers = 2
ff_hidden_size = 256
dropout = 0.2
use_avg_pool = True

best_test_loss = float('inf')
patience_counter = 0
best_path = "models/DeTention_best.pth"
final_path = "models/DeTention.pth"
scaler_path = "models/scaler.pkl"

# enforce divisibility constraints
if d_model % n_heads != 0:
    logger.info("d_model (%d) not divisible by n_heads (%d) -> resetting d_model to 64 and n_head to 4", d_model, n_heads)
    d_model = 64
    n_heads = 4

if ff_hidden_size & 1:
    logger.info("ff_hidden_size (%d) must be even for GLU -> resetting to 256", ff_hidden_size)
    ff_hidden_size = 256

Now, I need to get data in shape and format to start training:

In [11]:
df = get_data(ticker, period=period)
X_train, y_train, X_test, y_test, scaler = prepare_datasets(df, L=L, train_ratio=train_ratio)

# set torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(dim=-1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(dim=-1)

logger.info("Tensors -> X_train: %s, y_train: %s | X_test: %s, y_test: %s", X_train_t.shape, y_train_t.shape, X_test_t.shape, y_test_t.shape)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=batch_size, shuffle=False)

logger.info("DataLoaders -> train batches: %d, test batches: %d", len(train_loader), len(test_loader))

# Model & training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info("Using device: %s", device)

model = DeTention(seq_len=L, d_model=d_model, n_heads=n_heads, n_layers=n_layers, ff_hidden_size=ff_hidden_size, dropout=dropout, use_avg_pool=use_avg_pool).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

logger.info("Initialized DeTention -> layers: %d, d_model: %d, heads: %d", n_layers, d_model, n_heads)

21/12/2025 18:44:08 - BackToTheTrainLogger (Train) - INFO - HISTORY:
['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
                                Open       High        Low      Close  \
Date                                                                    
2022-12-20 00:00:00-05:00  23.910000  24.700001  23.719999  24.440001   
2022-12-21 00:00:00-05:00  24.510000  25.385000  24.139999  25.139999   
2022-12-22 00:00:00-05:00  24.559999  24.920000  24.080000  24.719999   
2022-12-23 00:00:00-05:00  24.520000  24.920000  24.299999  24.530001   
2022-12-27 00:00:00-05:00  24.200001  24.260000  23.410000  23.430000   

                             Volume  Dividends  Stock Splits  
Date                                                          
2022-12-20 00:00:00-05:00   7141600        0.0           0.0  
2022-12-21 00:00:00-05:00  10472500        0.0           0.0  
2022-12-22 00:00:00-05:00  11424100        0.0           0.0  
2022-12-23 00:00:00-05:00   5994

Time for training!

In [12]:
for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0.0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()

        pred = model(batch_x)
        loss = criterion(pred, batch_y.squeeze(dim=-1))

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * batch_x.size(0)

    train_loss /= len(train_loader.dataset)

    # evaluation
    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            pred = model(batch_x)
            loss = criterion(pred, batch_y.squeeze(dim=-1))

            test_loss += loss.item() * batch_x.size(0)

    test_loss /= len(test_loader.dataset)

    logger.info("Epoch %03d | Train Loss: %.4f | Test Loss: %.4f", epoch, train_loss, test_loss)

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        patience_counter = 0
        save_model(model, path=best_path)
        logger.info("New best model saved (test loss: %.4f)", best_test_loss)
    else:
        patience_counter += 1
        if patience_counter >= patience:
            logger.info("Early stopping triggered after epoch %d", epoch)
            break

21/12/2025 18:44:11 - BackToTheTrainLogger (Train) - INFO - Epoch 001 | Train Loss: 0.1574 | Test Loss: 0.0472
21/12/2025 18:44:11 - BackToTheTrainLogger (Train) - INFO - New best model saved (test loss: 0.0472)
21/12/2025 18:44:12 - BackToTheTrainLogger (Train) - INFO - Epoch 002 | Train Loss: 0.0478 | Test Loss: 0.0259
21/12/2025 18:44:12 - BackToTheTrainLogger (Train) - INFO - New best model saved (test loss: 0.0259)
21/12/2025 18:44:13 - BackToTheTrainLogger (Train) - INFO - Epoch 003 | Train Loss: 0.0193 | Test Loss: 0.0257
21/12/2025 18:44:13 - BackToTheTrainLogger (Train) - INFO - New best model saved (test loss: 0.0257)
21/12/2025 18:44:14 - BackToTheTrainLogger (Train) - INFO - Epoch 004 | Train Loss: 0.0159 | Test Loss: 0.0218
21/12/2025 18:44:14 - BackToTheTrainLogger (Train) - INFO - New best model saved (test loss: 0.0218)
21/12/2025 18:44:15 - BackToTheTrainLogger (Train) - INFO - Epoch 005 | Train Loss: 0.0146 | Test Loss: 0.0191
21/12/2025 18:44:15 - BackToTheTrainLogge

Metrics look promising, but can the hold up to expectations in `inference.ipynb`?

Saving the best model:

In [13]:
# load best (temporary) model and save as final, then remove temporary best
best_model = load_model(path=best_path)
save_model(best_model, path=final_path)
os.remove(best_path)

# Save the scaler for inference
joblib.dump(scaler, scaler_path)
logger.info("Training completed. Final model saved as '%s' (temporary best model removed)", final_path)
logger.info("Scaler saved as '%s'", scaler_path)

21/12/2025 18:44:59 - BackToTheTrainLogger (Train) - INFO - Training completed. Final model saved as 'models/DeTention.pth' (temporary best model removed)
21/12/2025 18:44:59 - BackToTheTrainLogger (Train) - INFO - Scaler saved as 'models/scaler.pkl'
